### Imports

In [9]:
import kmapper as km
from kmapper.plotlyviz import *
import sklearn
# DBSCAN from sklearn for clustering algorithms
from sklearn.cluster import DBSCAN
# PCA from sklearn for projection/lens creation
from sklearn.decomposition import PCA

# Plotly and Dash
import plotly.graph_objs as go
import dash_html_components as html
import dash_core_components as dcc
import dash
from ipywidgets import interactive, HBox, VBox, widgets, interact
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import pickle,json

### Functions

In [2]:
def cluster_fun(X, DBSCAN_EPSILON = 20, DBSCAN_MIN_SAMPLES = 1, N_CUBES = [7,7], PERC_OVERLAP = [.5,.5]):
    """

    """
    
    
#     keys = list(dict_df.keys())
#     print(keys)
#     X = dict_df.get(keys[0])
    # Killians interpolation naming for important columns
    kill = ["PREDICTED_WDP", "PREDICTED_SECCHI", "PREDICTED_TEMP", "PREDICTED_DO", 
           "PREDICTED_TURB","PREDICTED_COND", "PREDICTED_VEL", "PREDICTED_TP", 
           "PREDICTED_TN", "PREDICTED_SS", "PREDICTED_CHLcal"]
    
    # My interpolation naming for important columns
    my = ['PredictedTN','PredictedTP','TEMP','DO','TURB','COND','PredictedVEL','SS','WDP','CHLcal','SECCHI']
    nocond = ['PredictedTN','PredictedTP','TEMP','DO','TURB','PredictedVEL','SS','WDP','CHLcal','SECCHI']

    X = X[nocond]
    
    
    if X.shape[0]<DBSCAN_MIN_SAMPLES:
        #print(X)
        print("Not enough data to cluster in ", keys, "_size = ", X.shape[0])
        print("DBSCAN_MIN_SAMPLES", DBSCAN_MIN_SAMPLES)
        return([DBSCAN_MIN_SAMPLES, X.shape[0]])
    
    
    db = DBSCAN(eps=20, min_samples=2).fit(X)
    # Number of clusters in labels, ignoring noise if present.
    labels = db.labels_
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise_ = list(labels).count(-1)

    print('Estimated number of clusters: %d' % n_clusters_)
    print('Estimated number of noise points: %d' % n_noise_)
    
    return(db)

    
    
def mapper_pca_func(X,title, DBSCAN_EPSILON = 20, DBSCAN_MIN_SAMPLES = 1, N_CUBES = 7, PERC_OVERLAP = .5):
    """
    str(df_stratum_season_time_dict_list[1].keys())
    """
    
    # 
#     keys = list(dict_df.keys())
#     print(keys)
#     X = dict_df.get(keys[0])
    kill = ["PREDICTED_WDP", "PREDICTED_SECCHI", "PREDICTED_TEMP", "PREDICTED_DO", 
           "PREDICTED_TURB","PREDICTED_COND", "PREDICTED_VEL", "PREDICTED_TP", 
           "PREDICTED_TN", "PREDICTED_SS", "PREDICTED_CHLcal"]
    
    # My interpolation naming for important columns
    my = ['PredictedTN','PredictedTP','TEMP','DO','TURB','COND','PredictedVEL','SS','WDP','CHLcal','SECCHI']
    nocond = ['PredictedTN','PredictedTP','TEMP','DO','TURB','PredictedVEL','SS','WDP','CHLcal','SECCHI']

    X = X[nocond]
    
    X.reset_index(drop=True,inplace = True)
    
    var_to_index = {nocond[i] : i for i in range(len(nocond))}
    projected_vars = nocond
    projected_var_indices = [var_to_index[var] for var in projected_vars]
    
    print("X.shape: ",X.shape[0])
    
    if X.shape[0]<10:
        #print(X)
        print("Not enough data in ", keys, "_size = ", X.shape[0])
        return(X.shape[0])

    # defining clustering and kmapper parameters
    
    # create instance of clustering alg
    cluster_alg = sklearn.cluster.DBSCAN(eps=DBSCAN_EPSILON, min_samples=DBSCAN_MIN_SAMPLES, metric='euclidean')

    # Instantiate kepler mapper object
    mapper = km.KeplerMapper(verbose=0)
    
    # defining filter function as projection on to the first 2 component axis
    pca = PCA(n_components=1)
    lens = pca.fit_transform(X)
    print("Variance explained: ",pca.explained_variance_ratio_)
    
    # Project data onto the ones we want to display
    summary_variable = mapper.project(np.array(X),projection = projected_var_indices, scaler=None)
    
    # Generate the simplicial complex
    scomplex = mapper.map(lens, X, cover=km.Cover(n_cubes=N_CUBES, perc_overlap=PERC_OVERLAP), clusterer=cluster_alg,remove_duplicate_nodes=True)  


    pl_brewer = [[0.0, '#006837'],
             [0.1, '#1a9850'],
             [0.2, '#66bd63'],
             [0.3, '#a6d96a'],
             [0.4, '#d9ef8b'],
             [0.5, '#ffffbf'],
             [0.6, '#fee08b'],
             [0.7, '#fdae61'],
             [0.8, '#f46d43'],
             [0.9, '#d73027'],
             [1.0, '#a50026']]

    color_values = lens [:,0] - lens[:,0].min()
    my_colorscale = pl_brewer
    color_function_name = "Distance to x-min"
    kmgraph,  mapper_summary, colorf_distribution = get_mapper_graph(scomplex, 
                                                                     color_values,  
                                                                     color_function_name=color_function_name, 
                                                                     colorscale=my_colorscale)

    bgcolor = 'rgba(10,10,10, 0.9)'
    # y_gridcolor = 'rgb(150,150,150)'# on a black background the gridlines are set on  grey

    plotly_graph_data = plotly_graph(kmgraph, graph_layout='fr', colorscale=my_colorscale, 
                                     factor_size=2.5, edge_linewidth=0.5)
    plot_title = title + str(DBSCAN_EPSILON) + str(DBSCAN_EPSILON) + ', MIN_SAMPLES ' + str(DBSCAN_MIN_SAMPLES) 
    # plot_title = 'Pool 13, Summer 1993-1999; Epsilon ' + str(DBSCAN_EPSILON) + ', MIN_SAMPLES ' + str(DBSCAN_MIN_SAMPLES) 
    layout = plot_layout(title=plot_title,  
                         width=620, height=570,
                         annotation_text=get_kmgraph_meta(mapper_summary),  
                         bgcolor=bgcolor)
    
    # FigureWidget is responsible for event listeners

    fw_graph = go.FigureWidget(data=plotly_graph_data, layout=layout)
    fw_hist = node_hist_fig(colorf_distribution, bgcolor=bgcolor)
    fw_summary = summary_fig(mapper_summary, height=300)

    dashboard = hovering_widgets(kmgraph, 
                                 fw_graph, 
                                 bgcolor=bgcolor, 
                                 member_textbox_width=600)

    # DESIRED FILE PATH, CHANGE TO FIT YOUR LOCAL MACHINE
    directory_path = r"Mapper outputs\Conductivity Removed PCA 1"
    
    #Update the fw_graph colorbar, setting its title:
    fw_graph.data[1].marker.colorbar.title = 'dist to<br>x-min'
    html_output_path = directory_path + "\\" + title + 'PCA_1' + 'all_var_' + 'Eps_' + str(DBSCAN_EPSILON) +'MinS_' + str(DBSCAN_MIN_SAMPLES) + 'NCUBES_' + str(N_CUBES) + 'PEROvLp_' + str(PERC_OVERLAP) + '.html'
    html_output_path = html_output_path.replace(":","_")
    mapper.visualize(scomplex, path_html=html_output_path, color_values=color_values, color_function_name=color_function_name,lens=summary_variable,lens_names=projected_vars)
    return scomplex, X

### Load data

In [3]:
predicted_df = pd.read_csv(r"Interpolated Data\Open_river_interpolatedTN_TP_VEL.csv")
predicted_df.drop("Unnamed: 0",axis = 1,inplace = True)
continuous = ['PredictedTN','PredictedTP','TEMP','DO','TURB','COND','PredictedVEL','SS','WDP','CHLcal','SECCHI']
nocond = ['PredictedTN','PredictedTP','TEMP','DO','TURB','PredictedVEL','SS','WDP','CHLcal','SECCHI']
print("Initial interpolated size: ",predicted_df.shape[0])
print("Getting data TDA ready")
predicted_df.drop("COND",axis=1,inplace=True)
predicted_df.dropna(axis=0, how='any', thresh=None, subset=nocond, inplace=True)
print("TDA ready data size: ",predicted_df.shape[0])
predicted_df.head()

Initial interpolated size:  12381
Getting data TDA ready
TDA ready data size:  11314


,SHEETBAR,DATE,LATITUDE,LONGITUDE,FLDNUM,STRATUM,LOCATCD,TN,TP,TEMP,...,SS,WDP,CHLcal,SECCHI,YEAR,MONTH,SEASON,PredictedTN,PredictedTP,PredictedVEL
2,45001693,10/10/1995,37.288269,-89.514765,5,2,9553072,1.273,0.084,12.24140,...,25.6,5.0,12.24140,41.0,1995,10,FALL,1.273000,0.084000,0.01
4,45001698,10/10/1995,37.282954,-89.517829,5,2,9553149,NaN,NaN,11.99852,...,31.6,3.4,11.99852,40.0,1995,10,FALL,1.229465,0.103828,0.04
5,45001699,10/10/1995,37.276890,-89.510781,5,2,9553073,1.332,0.126,13.77964,...,35.1,1.3,13.77964,41.0,1995,10,FALL,1.332000,0.126000,0.31
8,45001702,10/10/1995,37.266244,-89.501127,5,2,9553074,1.302,0.085,7.70764,...,14.1,2.0,7.70764,50.0,1995,10,FALL,1.302000,0.085000,0.10
10,45001704,10/10/1995,37.245900,-89.498688,5,2,9553046,2.007,0.561,10.05548,...,49.3,2.8,10.05548,35.0,1995,10,FALL,2.007000,0.561000,0.02


In [4]:
### Creating three main time spans and two overlapping time spans,
### a total of five time spans

# defining different time periods
# first decade
time_dec1 = [1993, 1994, 1995, 1997, 1998, 1999, 2000]
# second decade
time_dec2 = [2001, 2002,2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013]
# third decade
time_dec3 = [2014, 2015, 2016, 2017, 2018, 2019, 2020]
# overlap time periods for continuity
time_overlap1 = [1998, 1999, 2000, 2001, 2002,2003, 2004]
time_overlap2 = [2010, 2011, 2012, 2013, 2014, 2015, 2016]
time_list = [time_dec1, time_overlap1, time_dec2, time_overlap2, time_dec3]

time_list_names = ['93-00', '98-04', '01-13', '10-16', '14-20']
# The stratums in my pool
stratum_list = [1, 2] #, 4, 5, 6, 7, 9]
# The seasons we are currently looking at
Season_names = ["SUMMER"] #, "SPRING", "FALL", "WINTER"]


df_stratum_season_time_dict = {}
s=""
for i in range(len(time_list)):
    for j in stratum_list:
        for k in Season_names:
            s= "Stratum " + str(j)+ " " + k + " " + time_list_names[i] + ": "
            df_stratum_season_time_dict[s] = predicted_df[(predicted_df['YEAR'].isin(time_list[i])) &
                                                             (predicted_df['STRATUM'].isin([j])) & 
                                                             (predicted_df['SEASON'].isin([k]))]
            s=""
print(len(df_stratum_season_time_dict))

10


In [5]:
for key in df_stratum_season_time_dict:
    print(key)

Stratum 1 SUMMER 93-00: 
Stratum 2 SUMMER 93-00: 
Stratum 1 SUMMER 98-04: 
Stratum 2 SUMMER 98-04: 
Stratum 1 SUMMER 01-13: 
Stratum 2 SUMMER 01-13: 
Stratum 1 SUMMER 10-16: 
Stratum 2 SUMMER 10-16: 
Stratum 1 SUMMER 14-20: 
Stratum 2 SUMMER 14-20: 


In [6]:
dbclus_dict = {}
for key in df_stratum_season_time_dict:
    dbclus_dict[key] = cluster_fun(df_stratum_season_time_dict[key])

Estimated number of clusters: 6
Estimated number of noise points: 12
Estimated number of clusters: 6
Estimated number of noise points: 15
Estimated number of clusters: 5
Estimated number of noise points: 13
Estimated number of clusters: 5
Estimated number of noise points: 16
Estimated number of clusters: 5
Estimated number of noise points: 12
Estimated number of clusters: 3
Estimated number of noise points: 10
Estimated number of clusters: 10
Estimated number of noise points: 25
Estimated number of clusters: 11
Estimated number of noise points: 17
Estimated number of clusters: 12
Estimated number of noise points: 32
Estimated number of clusters: 16
Estimated number of noise points: 26


In [7]:
n_cubes = [5, 10, 15]
perc_overlap = [.35, .45, .55]

mapper_output_dict = {}
mapper_output_df = {}
for key in df_stratum_season_time_dict:
    for j in n_cubes:
        for k in perc_overlap:
            newkey = key+"ncubes_"+str(j)+"_overlap_"+str(k)
            scomplex, X = mapper_pca_func(df_stratum_season_time_dict[key],key, N_CUBES = j, PERC_OVERLAP = k)
            mapper_output_df[newkey] = X
            mapper_output_dict[newkey] = scomplex

X.shape:  236
Variance explained:  [0.97162739]
X.shape:  236
Variance explained:  [0.97162739]
X.shape:  236
Variance explained:  [0.97162739]
X.shape:  236
Variance explained:  [0.97162739]
X.shape:  236
Variance explained:  [0.97162739]
X.shape:  236
Variance explained:  [0.97162739]
X.shape:  236
Variance explained:  [0.97162739]
X.shape:  236
Variance explained:  [0.97162739]
X.shape:  236
Variance explained:  [0.97162739]
X.shape:  255
Variance explained:  [0.96113053]
X.shape:  255
Variance explained:  [0.96113053]
X.shape:  255
Variance explained:  [0.96113053]
X.shape:  255
Variance explained:  [0.96113053]
X.shape:  255
Variance explained:  [0.96113053]
X.shape:  255
Variance explained:  [0.96113053]
X.shape:  255
Variance explained:  [0.96113053]
X.shape:  255
Variance explained:  [0.96113053]
X.shape:  255
Variance explained:  [0.96113053]
X.shape:  408
Variance explained:  [0.95382488]
X.shape:  408
Variance explained:  [0.95382488]
X.shape:  408
Variance explained:  [0.95

In [35]:
len(mapper_pca_output_dict)

90

Saving all outputs to json save scomplices and data

In [36]:

jsonFile = json.dumps(mapper_output_dict)
f = open("All_graphs.json","w")
pickle = pickle.dump(mapper_output_df,path)
f.write(jsonFile)
f.close()

Load mapper data

In [37]:
jsonFilePath = r"All_graphs.json"
jsonFile = open(jsonFilePath,"r")
data = json.load(jsonFile)
jsonFile.close()

In [38]:
len(data.keys())

90

Pick one output to analyze

In [43]:
#key = list(data.keys())[0]
key = "Stratum 2 SUMMER 01-13: ncubes_10_overlap_0.45"
print(key)
output = data[key]
output.keys()

Stratum 2 SUMMER 01-13: ncubes_10_overlap_0.45


dict_keys(['nodes', 'links', 'simplices', 'meta_data', 'meta_nodes'])

Function to return a nested list of connected components

In [127]:
def connected_components(output):
    simplices = output.get("simplices")
    # Pick out the edges
    pairs = [item for item in simplices if len(item)==2]
    # Pick out all nodes
    nodes = output.get("nodes").keys()
    # Unpacks the list of lists to get every node that is connected to at least one other node with an edge
    connected_nodes = [node for sublist in edges for node in sublist]
    # Pick out the nodes that are its own connected component (not a connected node)
    singles = [[node] for node in nodes if node not in connected_nodes]
    
    # Build connected components
    components = []
    for a, b in pairs:
        for component in components:
            if a in component:
                for i, other_component in enumerate(components):
                    if b in other_component and other_component != component: # a, and b are already in different components: merge
                        component.extend(other_component)
                        components[i:i+1] = []
                        break # we don't have to look for other components for b
                else: # b wasn't found in any other component
                    if b not in component:
                        component.append(b)
                break # we don't have to look for other components for a
            if b in component: # a wasn't in in the component 
                component.append(a)
                break # we don't have to look further
        else: # neither a nor b were found
            components.append([a, b])
            
    # Add the singles into the components
    print("Number of singles:",len(singles))
    components.extend(singles)
    
    # Sort the components by size
    components.sort(reverse = True, key = len)
    
    return components

In [128]:
components = connected_components(output)
len(components)
for component in components:
    print(len(component))

Number of singles: 15


In [130]:
components

[['cube0_cluster0',
  'cube1_cluster0',
  'cube2_cluster0',
  'cube2_cluster9',
  'cube3_cluster0',
  'cube3_cluster5',
  'cube3_cluster16',
  'cube4_cluster4',
  'cube5_cluster2'],
 ['cube3_cluster19',
  'cube4_cluster14',
  'cube3_cluster20',
  'cube5_cluster7',
  'cube6_cluster3'],
 ['cube5_cluster5',
  'cube6_cluster2',
  'cube7_cluster0',
  'cube8_cluster0',
  'cube9_cluster7'],
 ['cube4_cluster0', 'cube5_cluster0', 'cube6_cluster0', 'cube6_cluster1'],
 ['cube2_cluster17', 'cube3_cluster14', 'cube4_cluster12'],
 ['cube3_cluster11', 'cube4_cluster7', 'cube3_cluster12'],
 ['cube0_cluster1', 'cube1_cluster6'],
 ['cube0_cluster2', 'cube1_cluster11'],
 ['cube1_cluster1', 'cube2_cluster6'],
 ['cube1_cluster2', 'cube2_cluster7'],
 ['cube1_cluster3', 'cube2_cluster8'],
 ['cube1_cluster4', 'cube2_cluster11'],
 ['cube1_cluster5', 'cube2_cluster12'],
 ['cube1_cluster7', 'cube2_cluster13'],
 ['cube1_cluster8', 'cube2_cluster14'],
 ['cube1_cluster9', 'cube2_cluster15'],
 ['cube1_cluster10', 'c